In [4]:
import librosa
from librosa.display import specshow
import os
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt
import torch as torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import json
from paramManager import paramManager
from pathlib import Path
import soundfile as sf
import time

torch.__version__

'1.7.1'

In [5]:
audio_valid_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-valid/audio/'
labels_valid_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-valid/'

audio_train_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-train/audio/'
labels_train_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-train/'

audio_test_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-testid/audio/'
labels_test_dir = '/home/purnima/appdir/Github/DATA/NSynth/nsynth-test/'

filename = 'examples-subset-acoustic-synthetic-electronic-3000.json'
filename = 'examples.json'
filename = 'examples-subset-full-acoustic-3000.json'

In [6]:
test_df = pd.DataFrame()
with open(os.path.join(labels_test_dir, filename)) as f:
    params = json.load(f)
    test_df = pd.DataFrame.from_dict(params)
    test_df = test_df.transpose()

train_df = pd.DataFrame()
with open(os.path.join(labels_train_dir, filename)) as f:
    params = json.load(f)
    train_df = pd.DataFrame.from_dict(params)
    train_df = train_df.transpose()
    
valid_df = pd.DataFrame()
with open(os.path.join(labels_valid_dir, filename)) as f:
    params = json.load(f)
    valid_df = pd.DataFrame.from_dict(params)
    valid_df = valid_df.transpose()

In [7]:
instrument_list = ['keyboard','guitar','mallet','string','reed','brass','vocal','flute','bass']
total = sum(train_df.instrument_source_str.value_counts()) + sum(valid_df.instrument_source_str.value_counts()) + sum(test_df.instrument_source_str.value_counts())

print('Total instruments in train: ',sum(train_df.instrument_source_str.value_counts()),sum(train_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in valid: ',sum(valid_df.instrument_source_str.value_counts()),sum(valid_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in test: ',sum(test_df.instrument_source_str.value_counts()),sum(test_df.instrument_source_str.value_counts()) * 100/total)
print('----------------------------------')
print(train_df.instrument_source_str.value_counts())
print('----------------------------------')
print('Electronic breakdown:')
print(train_df[(train_df['instrument_source_str']=='electronic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Acoustic breakdown:')
print(train_df[(train_df['instrument_source_str']=='acoustic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Synthetic breakdown:')
print(train_df[(train_df['instrument_source_str']=='synthetic')].instrument_family_str.value_counts())

Total instruments in train:  24000 77.88920260928829
Total instruments in valid:  5124 16.62934475708305
Total instruments in test:  1689 5.481452633628663
----------------------------------
acoustic    24000
Name: instrument_source_str, dtype: int64
----------------------------------
Electronic breakdown:
Series([], Name: instrument_family_str, dtype: int64)
----------------------------------
Acoustic breakdown:
brass       3000
vocal       3000
guitar      3000
mallet      3000
string      3000
keyboard    3000
flute       3000
reed        3000
Name: instrument_family_str, dtype: int64
----------------------------------
Synthetic breakdown:
Series([], Name: instrument_family_str, dtype: int64)


In [8]:
print('Total instruments in train: ',sum(train_df.instrument_source_str.value_counts()),sum(train_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in valid: ',sum(valid_df.instrument_source_str.value_counts()),sum(valid_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in test: ',sum(test_df.instrument_source_str.value_counts()),sum(test_df.instrument_source_str.value_counts()) * 100/total)
print('----------------------------------')
print(test_df.instrument_source_str.value_counts())
print('----------------------------------')
print('Electronic breakdown:')
print(test_df[(test_df['instrument_source_str']=='electronic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Acoustic breakdown:')
print(test_df[(test_df['instrument_source_str']=='acoustic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Synthetic breakdown:')
print(test_df[(test_df['instrument_source_str']=='synthetic') ].instrument_family_str.value_counts())

Total instruments in train:  24000 77.88920260928829
Total instruments in valid:  5124 16.62934475708305
Total instruments in test:  1689 5.481452633628663
----------------------------------
acoustic    1689
Name: instrument_source_str, dtype: int64
----------------------------------
Electronic breakdown:
Series([], Name: instrument_family_str, dtype: int64)
----------------------------------
Acoustic breakdown:
guitar      481
string      306
brass       269
reed        235
mallet      202
keyboard    119
flute        55
vocal        22
Name: instrument_family_str, dtype: int64
----------------------------------
Synthetic breakdown:
Series([], Name: instrument_family_str, dtype: int64)


In [9]:
print('Total instruments in train: ',sum(train_df.instrument_source_str.value_counts()),sum(train_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in valid: ',sum(valid_df.instrument_source_str.value_counts()),sum(valid_df.instrument_source_str.value_counts()) * 100/total)
print('Total instruments in test: ',sum(test_df.instrument_source_str.value_counts()),sum(test_df.instrument_source_str.value_counts()) * 100/total)
print('----------------------------------')
print(valid_df.instrument_source_str.value_counts())
print('----------------------------------')
print('Electronic breakdown:')
print(valid_df[(valid_df['instrument_source_str']=='electronic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Acoustic breakdown:')
print(valid_df[(valid_df['instrument_source_str']=='acoustic')].instrument_family_str.value_counts())
print('----------------------------------')
print('Synthetic breakdown:')
print(valid_df[(valid_df['instrument_source_str']=='synthetic')].instrument_family_str.value_counts())

Total instruments in train:  24000 77.88920260928829
Total instruments in valid:  5124 16.62934475708305
Total instruments in test:  1689 5.481452633628663
----------------------------------
acoustic    5124
Name: instrument_source_str, dtype: int64
----------------------------------
Electronic breakdown:
Series([], Name: instrument_family_str, dtype: int64)
----------------------------------
Acoustic breakdown:
guitar      1482
brass        886
string       814
reed         720
mallet       663
keyboard     321
flute        155
vocal         83
Name: instrument_family_str, dtype: int64
----------------------------------
Synthetic breakdown:
Series([], Name: instrument_family_str, dtype: int64)


In [41]:
train_3k_acoustic_df = pd.DataFrame()
with open(os.path.join('/home/purnima/appdir/Github/DATA/NSynth/nsynth-train/', 'examples-subset-full-acoustic-3000.json')) as f:
    params = json.load(f)
    train_3k_acoustic_df = pd.DataFrame.from_dict(params)
    train_3k_acoustic_df = train_3k_acoustic_df.transpose()
    